In [72]:
import pandas as pd
import numpy as np
import glob

# GSDC Raw Data Overview

In [73]:
glob.glob("./data/raw_data/*")

['./data/raw_data/all_compounds_screened.csv',
 './data/raw_data/GDSC2_drug_dose_cellines_IC50s.xlsx',
 './data/raw_data/cellines_rnaseq_all_20220624',
 './data/raw_data/celline_SNP6_cnv_gistics_20191101',
 './data/raw_data/all_cellines_screened.xlsx',
 './data/raw_data/cellines_rnaseq_all_20220624.zip',
 './data/raw_data/mutations_all_20230202.csv',
 './data/raw_data/mutations_summary_20230202.csv',
 './data/raw_data/cellines_mutations_summary_20230202.zip',
 './data/raw_data/celline_SNP6_cnv_gistics_20191101.zip',
 './data/raw_data/drug_info.csv',
 './data/raw_data/cellines_mutations_all_20230202.zip']

In [74]:
all_compounds_screened = pd.read_csv("./data/raw_data/all_compounds_screened.csv")
all_cellines_screened = pd.read_excel("./data/raw_data/all_cellines_screened.xlsx", sheet_name=0)
all_experiment = pd.read_excel("./data/raw_data/GDSC2_drug_dose_cellines_IC50s.xlsx")

/Users/sunzehui/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [75]:
# 选取有测序结果和甲基化数据等数据的细胞系
filtered_celline = all_cellines_screened.loc[
    (all_cellines_screened['Whole Exome Sequencing (WES)'] == "Y") &
    (all_cellines_screened['Methylation'] == "Y") &
    (all_cellines_screened['Gene Expression'] == "Y") &
    (all_cellines_screened['Copy Number Alterations (CNA)'] == "Y") &
    (all_cellines_screened['Drug\nResponse'] == "Y")
]
filtered_celline.head()

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [76]:
filtered_celline.head()

,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [77]:
a = set(all_experiment['CELL_LINE_NAME'])
b = set(filtered_celline['Sample Name'])
print(f"experiment sheet includes {a.__len__()} unqiue cellines")
print(f"all_celline sheet includes {b.__len__()} unique cellines")
celline_barcode = list(a.intersection(b))
print(f"two sheets have {celline_barcode.__len__()} overlapping cellines")

experiment sheet includes 969 unqiue cellines
all_celline sheet includes 928 unique cellines
two sheets have 870 overlapping cellines


In [78]:
filtered_experiment = all_experiment[all_experiment['CELL_LINE_NAME'].isin(celline_barcode)]
filtered_experiment.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


# Single Celline Multi-omics data Overview

## Copy Number Variation

In [79]:
all_cna = pd.read_csv("./data/raw_data/celline_SNP6_cnv_gistics_20191101/cnv_abs_copy_number_picnic_20191101.csv")
all_fpkm = pd.read_csv("./data/raw_data/cellines_rnaseq_all_20220624/rnaseq_fpkm_20220624.csv")

/var/folders/wc/rh6kv76j5dxdxqbbrkkc3bn80000gn/T/ipykernel_40185/1966536984.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [80]:
# Doing some preprocessing
dat = all_cna.iloc[2:24505]
titile = all_cna.iloc[[0]]
df = pd.concat([titile, dat])
df = df.T.iloc[1:]
new_header = df.iloc[[0]].values
new_header = new_header.tolist()
new_header = new_header[0]
new_header[0] = "celline_barcode"
df.columns = new_header
df = df.iloc[1:].set_index("celline_barcode")

In [81]:
df = df.fillna(0.0)
cna_df = df

## Gene Expression (FPKM-based)

In [82]:
dat = all_fpkm.iloc[4:]
titile = all_fpkm.iloc[[0]]
df = pd.concat([titile, dat])
df = df.T.iloc[1:]
df
new_header = df.iloc[[0]].values
new_header = new_header.tolist()
new_header = new_header[0]
new_header[0] = "celline_barcode"
df.columns = new_header
df = df.iloc[1:].set_index("celline_barcode")

In [83]:
df = df.fillna(0.0)
fpkm_df = df

## Datasets Alignment

In [84]:
s1 = set(fpkm_df.index).intersection(celline_barcode)
s2 = set(cna_df.index).intersection(celline_barcode)
celline_barcode = list(s1.intersection(s2))
print(f"Two datasets have {len(celline_barcode)} common cellines")

Two datasets have 866 common cellines


In [85]:
common_genes = list(set(cna_df.columns).intersection(fpkm_df.columns))
print(f"Two datasets have {len(common_genes)} genes(features)")

Two datasets have 23914 genes(features)


In [86]:
fpkm_df.loc[celline_barcode].to_csv("./data/processed_data/fpkm.csv", sep='\t')
cna_df.loc[celline_barcode].to_csv("./data/processed_data/cna.csv", sep='\t')
fpkm_df = fpkm_df.loc[celline_barcode]
cna_df = cna_df.loc[celline_barcode]

In [87]:
filtered_experiment[filtered_experiment['CELL_LINE_NAME'].isin(celline_barcode)].head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


## GDSC Dataset

Filter dataset

In [88]:
# drug_info: https://www.cancerrxgene.org/downloads/drug_data
drug_df = pd.read_csv('./data/raw_data/drug_info.csv', sep=',')
drug_df.head()

,drug_id,drug_name,synonyms,pathway_name,targets,pubchem
0,1834,743380,-,Unclassified,-,-
1,1854,MN-64,-,WNT signaling,"TNKS1, TNKS2",2802462
2,1909,Venetoclax,"ABT-199, Veneclexta, GDC-0199",Apoptosis regulation,BCL2,49846579
3,1913,AGI-5198,IDH-C35,Metabolism,IDH1 (R132H),56645356
4,1915,AZD3759,-,EGFR signaling,EGFR,78209992


In [89]:
# pubchem_id exclude non-numeric rows
import re
nonnumber = re.compile(r'\D+')
pubchem_id = list(set(drug_df['pubchem']))
pubchem_id = [i.split(',')[0] if "," in i else i for i in pubchem_id]
pubchem_id = [i for i in pubchem_id if re.findall(pattern=nonnumber, string=i).__len__()==0]
drug_df = drug_df[drug_df['pubchem'].isin(pubchem_id)]

In [90]:
all_drug_in_experiment = list(set(all_experiment['DRUG_NAME']))
d = drug_df[drug_df['drug_name'].isin(all_drug_in_experiment)][['drug_name', 'pubchem']]

In [91]:
lookup_table = dict(zip(d['drug_name'], d['pubchem']))
all_experiment = all_experiment[all_experiment['DRUG_NAME'].isin(d['drug_name'])]
all_experiment['PUBCHEM_ID'] = [lookup_table[i] for i in all_experiment['DRUG_NAME']]


In [92]:
all_experiment = all_experiment[all_experiment['CELL_LINE_NAME'].isin(celline_barcode)]
all_experiment.reset_index(inplace = True)
all_experiment.drop(columns = "index", inplace = True)
all_experiment

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,PUBCHEM_ID
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.462148,0.930105,0.088999,0.432482,24360
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869447,0.614932,0.111423,-1.420322,24360
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360684,0.790953,0.142754,-0.599894,24360
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.045014,0.592624,0.135642,-1.515791,24360
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741620,0.733992,0.128066,-0.807038,24360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145002,GDSC2,401,19186874,1659818,MM1S,SIDM01265,MM,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,4.290123,0.991533,0.020244,-0.962882,70983932
145003,GDSC2,401,19187483,1659928,SNU-175,SIDM00216,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.423870,0.985066,0.032710,0.001082,70983932
145004,GDSC2,401,19187936,1660034,SNU-407,SIDM00214,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,5.042005,0.970009,0.026729,-0.323597,70983932
145005,GDSC2,401,19188194,1660035,SNU-61,SIDM00194,COREAD,2359,GSK2830371,WIP1,Other,1094,Y,0.010005,10.0,6.130028,0.986298,0.027488,0.601490,70983932


In [93]:
all_experiment.to_csv("./data/processed_data/expriment.csv", sep="\t", index = None)

## Response and Pubchem ID troubleshooting

In [94]:
import pubchempy as pcp
df = pcp.get_properties(properties=['canonical_smiles'], identifier=pubchem_id,
                        namespace='cid', )
df = pd.DataFrame(df)
df[['CID']]=df[['CID']].astype(str)
df.to_csv("./data/processed_data/pubchem_id-SMILES.csv", sep='\t')
lookup_table_cid_smiles = dict(zip(df['CID'], df['CanonicalSMILES']))

In [95]:
all_experiment = all_experiment[all_experiment['PUBCHEM_ID'].isin(pubchem_id)]
all_experiment['SMILES']=[lookup_table_cid_smiles[i] for i in all_experiment['PUBCHEM_ID']]

In [96]:
all_experiment.head(5)

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,...,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,PUBCHEM_ID,SMILES
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,...,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038,24360,CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=CC=CC=C5N=C4C3...


In [97]:
sample_barcode = [f"{i[0]}_{i[1]}" for i in zip(all_experiment['CELL_LINE_NAME'], all_experiment['PUBCHEM_ID'])]

In [98]:
response = pd.DataFrame()
response['sample_barcode'] = sample_barcode
response['LN_IC50'] = all_experiment['LN_IC50']
response.reset_index(drop=True, inplace=True)

In [99]:
response.to_csv('./data/processed_data/response.csv', sep='\t', index=None)
all_experiment.to_csv('./data/processed_data/expriment.csv', sep='\t', index=None)

## Drug Feature Extraction using beta-VAE

In [100]:
import pandas as pd
from os.path import realpath
import pandas as pd
from keras.models import load_model
import ast
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import BondType

RDLogger.DisableLog("rdApp.*")

SMILE_CHARSET = '["C", "B", "F", "I", "H", "O", "N", "S", "P", "Cl", "Br"]'

bond_mapping = {"SINGLE": 0, "DOUBLE": 1, "TRIPLE": 2, "AROMATIC": 3}
bond_mapping.update(
    {0: BondType.SINGLE, 1: BondType.DOUBLE, 2: BondType.TRIPLE, 3: BondType.AROMATIC}
)
SMILE_CHARSET = ast.literal_eval(SMILE_CHARSET)

SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)

BATCH_SIZE = 100
EPOCHS = 10

VAE_LR = 5e-4
NUM_ATOMS = 120  # Maximum number of atoms

ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 435  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    # Ignore Pt Atom
    for atom in molecule.GetAtoms():
        if(atom.GetSymbol() == "Pt"):
            continue
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule


# 注：min-Max归一化需要在分割完训练集和测试集和Validation set之后再进行

# Cached_data
cached_data = {}

# Path Define
cna_path = realpath("data/processed_data/cna.csv")
experiment_path = realpath("data/processed_data/experiment.csv")
fpkm_path = realpath("data/processed_data/fpkm.csv")
SMILES_path = realpath("data/processed_data/pubchem_id-SMILES.csv")

drug_AdjacencyTensor = []
drug_FeatureTensor = []

df = pd.read_csv(SMILES_path, sep='\t')
for i in df["CanonicalSMILES"]:
    _ad, _fe = smiles_to_graph(i)
    drug_AdjacencyTensor.append(_ad)
    drug_FeatureTensor.append(_fe)

drug_AdjacencyTensor = np.array(drug_AdjacencyTensor)
drug_FeatureTensor = np.array(drug_FeatureTensor)

vae = load_model("./model/drug-molecule-generation-with-VAE",
               compile=False)

z_mean, _ = vae.encoder.predict([drug_AdjacencyTensor[:1000], drug_FeatureTensor[:1000]])

6/6 [==============================] - 0s 3ms/step


In [101]:
drug_feature_df = pd.DataFrame(data = z_mean, index=df['CID'])

In [102]:
drug_feature_df.head(5) # 435-dim vector

,0,1,2,3,4,5,6,7,8,9,...,425,426,427,428,429,430,431,432,433,434
CID,,,,,,,,,,,,,,,,,,,,,
24771867,0.005877,0.075289,-0.166716,0.001400,-0.021318,-0.002469,0.155582,-0.043718,-0.407758,0.065322,...,0.017054,-0.146396,-0.228857,-0.046832,0.016400,-0.117910,0.111531,-0.207286,0.110159,-0.060695
44631912,0.041470,0.098258,-0.174645,0.007417,0.005180,-0.014222,0.137256,-0.020531,-0.390423,0.051803,...,0.013894,-0.130984,-0.228654,-0.040384,0.011722,-0.101904,0.101874,-0.179096,0.103770,-0.059894
24776445,0.005402,0.045830,-0.131910,0.058264,-0.074162,0.021043,0.151301,-0.050268,-0.393139,0.119950,...,-0.014712,-0.129678,-0.244254,-0.052003,0.012882,-0.095890,0.072875,-0.193941,0.102836,-0.066884
25227847,0.041659,0.102329,-0.189710,-0.018375,0.018527,-0.010759,0.125587,-0.020467,-0.389242,0.024223,...,0.017338,-0.133892,-0.215924,-0.038210,0.014984,-0.100185,0.106379,-0.174425,0.098170,-0.058998
11655119,0.011208,0.085992,-0.183917,0.002001,-0.018637,-0.004944,0.168668,-0.044072,-0.443718,0.071907,...,0.016255,-0.160898,-0.251479,-0.049126,0.017321,-0.131497,0.119796,-0.223002,0.120605,-0.066833


## CNA 和 Gene-Expression dimension reduction

In [103]:
print(cna_df.shape)
cna_df.head()

(866, 24502)


,A1BG,A1CF,A2M,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZZEF1,ZZZ3
celline_barcode,,,,,,,,,,,,,,,,,,,,,
NCI-H28,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,2.0,2.0,3.0,5.0,5.0,5.0,4.0,3.0,5.0
KS-1,4.0,4.0,3.0,3.0,3.0,3.0,3.0,2.0,3.0,4.0,...,4.0,2.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,3.0
BT-474,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,4.0,3.0,...,3.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0
PF-382,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
NCI-H2029,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,2.0,3.0


In [104]:
print(fpkm_df.shape)
fpkm_df.head()

(866, 37602)


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A2MP1,A3GALT2,...,ZYG11A,ZYG11AP1,ZYG11B,ZYX,ZYXP1,ZZEF1,ZZZ3,RSKR,SHLD3,ATP6V1FNB
celline_barcode,,,,,,,,,,,,,,,,,,,,,
NCI-H28,0.01,0.12,0.0,0.01,0.17,0.01,0.0,0.48,0.0,0.1,...,0.03,0.0,2.88,28.06,0.0,4.37,8.93,2.71,1.75,5.62
KS-1,0.38,2.25,0.0,20.06,0.0,0.0,0.0,0.43,0.0,0.0,...,0.01,0.0,3.37,107.99,0.0,4.34,5.56,1.25,1.23,0.82
BT-474,0.02,0.05,0.08,0.0,0.39,0.47,0.0,0.66,0.34,0.04,...,0.73,0.0,4.12,14.62,0.0,4.67,5.68,1.97,1.69,0.12
PF-382,0.44,0.68,0.07,0.14,0.03,0.01,0.0,1.42,0.12,0.06,...,0.04,0.0,2.09,4.84,0.0,11.48,4.30,8.35,2.38,0.28
NCI-H2029,0.23,1.2,0.01,0.0,0.0,0.0,0.0,1.34,0.0,0.11,...,0.53,0.0,13.42,0.84,0.0,5.03,15.11,3.71,3.52,1.43


**CNA and FPKM data integration using SNF**

SNFtool: https://doi.org/10.1038/nmeth.2810

R code is in current folder/SNL_integration.R

In [105]:
!R CMD BATCH ./SNL_integration.R

In [106]:
similarity_df = pd.read_csv("./data/processed_data/simlilarity_matrix.csv")

In [107]:
similarity_df.drop(columns=['Unnamed: 0'], inplace=True)
similarity_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V857,V858,V859,V860,V861,V862,V863,V864,V865,V866
0,0.560332,0.001149,0.000815,0.001148,0.000878,0.001141,0.001118,0.000940,0.000920,0.001099,...,0.002365,0.000956,0.001000,0.001172,0.001064,0.001229,0.001019,0.000900,0.001180,0.001077
1,0.001149,0.553651,0.000850,0.001773,0.000936,0.001272,0.000958,0.000909,0.001011,0.001168,...,0.001007,0.001283,0.000959,0.001029,0.000952,0.000958,0.001654,0.000896,0.001086,0.001202
2,0.000815,0.000850,0.580935,0.000764,0.000812,0.000859,0.000735,0.000923,0.001197,0.000746,...,0.000747,0.000938,0.000813,0.000900,0.001104,0.001812,0.000893,0.000852,0.000806,0.000831
3,0.001148,0.001773,0.000764,0.555381,0.000809,0.000930,0.001045,0.000878,0.000794,0.001199,...,0.001049,0.001004,0.000834,0.000983,0.000832,0.000923,0.000935,0.000869,0.000966,0.001064
4,0.000878,0.000936,0.000812,0.000809,0.603372,0.000926,0.000856,0.001385,0.000885,0.001136,...,0.000953,0.000944,0.001358,0.000935,0.001419,0.000931,0.000887,0.000852,0.000900,0.001282


In [108]:
similarity_df.columns = cna_df.index

In [109]:
similarity_df.head() # 就是细胞的Genomic Profiles的特征向量
celline_feature = {}
for i, celline in enumerate(similarity_df.columns):
    celline_feature[celline] = np.array(similarity_df.iloc[i].values)

In [110]:
s = []
for i in response['sample_barcode']:
    celline_name, pubchem_id = i.split('_')
    celline_feature_array = celline_feature[celline_name]
    drug_feature_array = drug_feature_df.loc[int(pubchem_id)]
    combined_feature = np.hstack([celline_feature_array, drug_feature_array])
    s.append(combined_feature)
    

In [111]:
np.array(s).shape # 866+435=1301-dim vector

(145007, 1301)

In [112]:
np.save("./data/processed_data/all_feature", np.array(s))